In [69]:
import re
import os
import io
import sys
import subprocess
from dotenv import load_dotenv
from openai import OpenAI
import google.generativeai
import anthropic
import execjs
import gradio as gr

# Load environment variables
load_dotenv()
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY', 'your-key-if-not-using-env')
os.environ['ANTHROPIC_API_KEY'] = os.getenv('ANTHROPIC_API_KEY', 'your-key-if-not-using-env')
os.environ['GOOGLE_API_KEY'] = os.getenv('GOOGLE_API_KEY','your-key-if-not-using-env')
# Initialize API Clients
openai = OpenAI()
claude = anthropic.Anthropic()
google.generativeai.configure()
# Default AI Models
OPENAI_MODEL = "gpt-4o"
CLAUDE_MODEL = "claude-3-5-sonnet-20240620"
GEMINI_MODEL ='gemini-1.5-flash'

In [70]:
def optimize(input_code, output_code, model):
    """Optimizes input code and converts it to the desired output language using AI."""
    user_prompt = (
        f"Rewrite this {input_code} code in C++ with the fastest possible implementation that produces identical output in the least time. "
        f"Respond only with {output_code} code; do not explain your work other than a few comments. "
        f"Pay attention to number types to ensure no int overflows. Remember to #include all necessary {output_code} packages such as iomanip.\n\n"
        f"{input_code}"
    )
    
    system_message = (
        f"You are an assistant that reimplements {input_code} code in high-performance {output_code}. "
        f"Respond only with {output_code} code; use comments sparingly and do not provide any explanation other than occasional comments. "
        f"The {output_code} response needs to produce an identical output in the fastest possible time."
    )

    if model == "GPT":
        result = stream_gpt(system_message, user_prompt)
    elif model == "Claude":
        result = stream_claude(system_message, user_prompt)
    elif model == "Gemini":
        result = stream_gemini(system_message, user_prompt)
    else:
        raise ValueError("Unknown model")
    
    return result

In [74]:

def write_output(cpp):
    """Removes markdown syntax from C++ code blocks."""
    return re.sub(r"```(cpp)?", "", cpp).strip()

def stream_gpt(system_message, user_prompt): 
    """Streams output from OpenAI GPT model."""
    stream = openai.chat.completions.create(
        model=OPENAI_MODEL, 
        messages=[{"role": "system", "content": system_message}, {"role": "user", "content": user_prompt}], 
        stream=True
    )
    reply = ""
    for chunk in stream:
        fragment = chunk.choices[0].delta.content or ""
        reply += fragment
        print(fragment, end='', flush=True)

    return write_output(reply)
import google.generativeai as genai

def stream_gemini(system_message, user_prompt):
    """Streams output from Google Gemini model."""
    gemini = genai.GenerativeModel(
        model_name=GEMINI_MODEL,
        system_instruction=system_message
    )
    response_stream = gemini.generate_content(user_prompt)
    reply = ""
    for chunk in response_stream:
        fragment = chunk.text or ""
        reply += fragment
        print(fragment, end='', flush=True) 
    return write_output(reply)  

def stream_claude(system_message, user_prompt):
    """Streams output from Claude AI model."""
    response = claude.messages.create(
        model=CLAUDE_MODEL,
        max_tokens=2000,
        system=system_message,
        messages=[{"role": "user", "content": user_prompt}]
    )

    reply = ""
    for text in response.text_stream:
        reply += text
        print(text, end="", flush=True)

    return write_output(reply)

In [75]:
def runthecode(language, code):
    """Runs the given code based on the specified language."""
    if language == 'C++':
        return execute_cpp(code)
    elif language == 'Python':
        return execute_python(code)
    elif language == 'JavaScript':
        return execute_javascript(code)
    else:
        return "Unsupported language"

def execute_cpp(code):
    """Compiles and runs C++ code."""
    try:
        with open("optimized.cpp", "w") as file:
            file.write(code)

        compile_cmd = ["clang++", "-Ofast", "-std=c++17", "-march=armv8.5-a", "-mtune=apple-m1", "-mcpu=apple-m1", "-o", "optimized", "optimized.cpp"]
        subprocess.run(compile_cmd, check=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)

        run_cmd = ["./optimized"]
        run_result = subprocess.run(run_cmd, check=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
        
        return run_result.stdout
    except subprocess.CalledProcessError as e:
        return f"An error occurred:\n{e.stderr}"

def execute_python(code):
    """Executes Python code and captures output."""
    output = io.StringIO()
    sys.stdout = output
    try:
        exec(code, {}, {})
    finally:
        sys.stdout = sys.__stdout__
    return output.getvalue()

def execute_javascript(js_code):
    """Executes JavaScript code using Node.js."""
    try:
        output = execjs.eval(js_code)
        return str(output)
    except Exception as e:
        return f"Error: {str(e)}"

In [76]:


# CSS for UI
css = """
.inputcode {background-color: #306998;}
.outputcode {background-color: #050;}
"""

# Gradio UI
with gr.Blocks(css=css) as ui:
    gr.Markdown("## Convert code from Python to C++")
    gr.Markdown("### You cannot have both input and output codes as the same")

    with gr.Row():
        input_code = gr.Dropdown(['JavaScript', 'Python', 'C++'], label="Given Language Code", value="Python")
        output_code = gr.Dropdown(['JavaScript', 'Python', 'C++'], label="Convert To", value="C++")

    with gr.Row():
        inputs = gr.Textbox(label="Input Code:", lines=10)
        outputs = gr.Textbox(label="Converted Code:", lines=10)

    with gr.Row():
        model = gr.Dropdown(["GPT", "Claude","Gemini"], label="Select Model", value="GPT")

    with gr.Row():
        convert = gr.Button("Convert Code")

    with gr.Row():
        input_run = gr.Button("Run Input Code")
        output_run = gr.Button("Run Output Code")

    with gr.Row():
        inputs_compiler = gr.TextArea(label="Input Result:", elem_classes=["inputcode"])
        outputs_compiler = gr.TextArea(label="Output Result:", elem_classes=["outputcode"])

    # Button event handlers
    convert.click(optimize, inputs=[inputs, output_code, model], outputs=[outputs])
    input_run.click(runthecode, inputs=[input_code, inputs], outputs=[inputs_compiler])
    output_run.click(runthecode, inputs=[output_code, outputs], outputs=[outputs_compiler])

ui.launch(inbrowser=True)

Traceback (most recent call last):
  File "/opt/anaconda3/envs/llms/lib/python3.11/site-packages/gradio/queueing.py", line 625, in process_events
    response = await route_utils.call_process_api(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/anaconda3/envs/llms/lib/python3.11/site-packages/gradio/route_utils.py", line 322, in call_process_api
    output = await app.get_blocks().process_api(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/anaconda3/envs/llms/lib/python3.11/site-packages/gradio/blocks.py", line 2044, in process_api
    result = await self.call_function(
             ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/anaconda3/envs/llms/lib/python3.11/site-packages/gradio/blocks.py", line 1591, in call_function
    prediction = await anyio.to_thread.run_sync(  # type: ignore
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/anaconda3/envs/llms/lib/python3.11/site-packages/anyio/to_thread.py", line 56, in run_sync
    return